### Load modules

In [1]:
import pandas as pd
import numpy as np
import os, re 


In [22]:
current_subject = 'sub007'

position_names_file        = '../data/01_gaze_data/' + current_subject + '_billboard_list_position_name_assignment.csv'
passing_viewing_list_file  = '../data/01_gaze_data/' + current_subject + '_lists_1.txt'
recall_file                = '../data/03_interview_qualtrics_data/recall_sheet.csv'
recognition_file           = '../data/03_interview_qualtrics_data/recognition_sheet.csv'

#### Reading the position_name file

this gives us the "raw names" of which billboards were for this ppt at which position, which will be used to relabel the output files.

In [23]:
df_position_names  = pd.read_csv(position_names_file)
df_position_names['billboard_name_at_position_cleaned'] = df_position_names['billboard_image_at_position'].apply(lambda x: x.replace("resources/billboards/", "").replace(".bmp", ""))
df_dropped = df_position_names.drop(['index', 'position_index', 'billboard_image_at_position'], axis=1)
mapping_dict = pd.Series(df_dropped['billboard_name_at_position_cleaned'].values, index=df_dropped['billboard_named_position']).to_dict()

#### Reading the file with the remembered billboards

This gives us the behavioral information, i.e. which bbs were looked at, not looked at, incepted, etc.

In [20]:
with open(passing_viewing_list_file, 'r', encoding='utf-8') as file:
    data = file.read()


# Regular expressions to identify the different sections
section_regex = r"(Participant ID:\s\w+|passed_billboard_index_list:|passed_fixated_first_list:|passed_fixated_incepted_list:|repeated_forgone_list:|forgone_list:|passed_list:|glanced_list:)"
billboard_regex = r"billboard_image_\d+"

# Parsing function
def parse_data(data):
    parsed_data = {}
    current_section = None
    for line in data.splitlines():
        # Check if the line starts a new section
        section_match = re.match(section_regex, line)
        if section_match:
            current_section = section_match.group().strip(':')
            parsed_data[current_section] = []
        elif current_section:  # Add billboard data to the current section
            billboards = re.findall(billboard_regex, line)
            if billboards:
                parsed_data[current_section].extend(billboards)
    return parsed_data


parsed_data = parse_data(data)

# Convert to a DataFrame (optional)
df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in parsed_data.items()]))

##

passed_fixated_first_list = df['passed_fixated_first_list'].dropna()
passed_fixated_first_list = list(passed_fixated_first_list.values)
passed_fixated_first_list = [mapping_dict.get(item, item) for item in passed_fixated_first_list]
#print(passed_fixated_first_list)

passed_fixated_incepted_list = df['passed_fixated_incepted_list'].dropna()
passed_fixated_incepted_list = list(passed_fixated_incepted_list.values)
passed_fixated_incepted_list = [mapping_dict.get(item, item) for item in passed_fixated_incepted_list]
#print(passed_fixated_incepted_list)

repeated_forgone_list = df['repeated_forgone_list'].dropna()
repeated_forgone_list = list(repeated_forgone_list.values)
repeated_forgone_list = [mapping_dict.get(item, item) for item in repeated_forgone_list]
#print(repeated_forgone_list)

forgone_list = df['forgone_list'].dropna()
forgone_list = list(forgone_list.values)
forgone_list = [mapping_dict.get(item, item) for item in forgone_list]
#print(forgone_list)

passed_list = df['passed_list'].dropna()
passed_list = list(passed_list.values)
passed_list = [mapping_dict.get(item, item) for item in passed_list]
#print(passed_list)

glanced_list = df['glanced_list'].dropna()
glanced_list = list(glanced_list.values)
glanced_list = [mapping_dict.get(item, item) for item in glanced_list]
#print(glanced_list)


#### Reading the recall data file

this gives us this participant's memory

In [21]:
recall_df = pd.read_csv(recall_file)
this_sub_df = recall_df[recall_df['subject'] == current_subject]
this_sub_recalled = this_sub_df.columns[this_sub_df.iloc[0] == 1].tolist()
this_sub_recalled

['great_lakes', 'polar_bear', 'vaccination', 'carpet_cleaning', 'cosmetics']

#### Combining the viewing lists with the recalls

In [6]:
# passed_fixated_first_list
# determine which items from the recall_list where in the passed_fixated_first_list
common_elements = [item for item in passed_fixated_first_list if item in this_sub_recalled]
#print(common_elements) 

print('recall rate passed_fixated')
recall_passed_fixated_first = len(common_elements)/len(passed_fixated_first_list) if len(passed_fixated_first_list) != 0 else 0
print(recall_passed_fixated_first)


# passed_fixated_incepted_list
# determine which items from the recall_list where in the passed_fixated_incepted_list
common_elements = [item for item in passed_fixated_incepted_list if item in this_sub_recalled]
#print(common_elements) 

print('recall rate fixated incepted')
recall_fixated_incepted = len(common_elements)/len(passed_fixated_incepted_list) if len(passed_fixated_incepted_list) != 0 else 0
print(recall_fixated_incepted)


# repeated_forgone_list
# determine which items from the recall_list where in the repeated_forgone_list
common_elements = [item for item in repeated_forgone_list if item in this_sub_recalled] 
#print(common_elements) 

print('recall rate repeated forgone')
recall_repeated_forgone = len(common_elements)/len(repeated_forgone_list) if len(repeated_forgone_list) != 0 else 0
print(recall_repeated_forgone)


# forgone_list
# determine which items from the recall_list where in the forgone_list
common_elements = [item for item in forgone_list if item in this_sub_recalled] 
#print(common_elements) 

print('recall rate forgone')
recall_forgone = len(common_elements)/len(forgone_list) if len(forgone_list) != 0 else 0
print(recall_forgone)


recall rate passed_fixated
0.1
recall rate fixated incepted
1.0
recall rate repeated forgone
0
recall rate forgone
0.0


#### Reading the Recognition Data

In [7]:
recognition_df = pd.read_csv(recognition_file)
this_sub_df = recognition_df[recognition_df['subject'] == current_subject]
this_sub_recognized = this_sub_df.columns[this_sub_df.iloc[0] == 1].tolist()
this_sub_recognized

['binge_drinking',
 'buckle_up',
 'caffeine_consumption',
 'diabetes_awareness',
 'regular_doctor_visits',
 'drug_abuse',
 'employee_wellness',
 'healthy_diet',
 'HIV',
 'jogging_physical_activity',
 'mental_health',
 'polar_bear',
 'recycling',
 'sun_skin_protection',
 'burger',
 'car_insurance',
 'carpet_cleaning',
 'clothes_sale',
 'coffee',
 'fundraising',
 'pet_store',
 'pizza',
 'pottery_class']

In [8]:
# passed_fixated_first_list
# determine which items from the recall_list where in the passed_fixated_first_list
common_elements = [item for item in passed_fixated_first_list if item in this_sub_recognized]
#print(common_elements) 

print('recognition rate passed_fixated')
recognition_passed_fixated_first = len(common_elements)/len(passed_fixated_first_list) if len(passed_fixated_first_list) != 0 else 0
print(recognition_passed_fixated_first)


# passed_fixated_incepted_list
# determine which items from the recall_list where in the passed_fixated_incepted_list
common_elements = [item for item in passed_fixated_incepted_list if item in this_sub_recognized]
#print(common_elements) 

print('recognition rate fixated incepted')
recognition_fixated_incepted = len(common_elements)/len(passed_fixated_incepted_list) if len(passed_fixated_incepted_list) != 0 else 0
print(recognition_fixated_incepted)


# repeated_forgone_list
# determine which items from the recall_list where in the repeated_forgone_list
common_elements = [item for item in repeated_forgone_list if item in this_sub_recognized] 
#print(common_elements) 

print('recognition rate repeated forgone')
recognition_repeated_forgone = len(common_elements)/len(repeated_forgone_list) if len(repeated_forgone_list) != 0 else 0
print(recognition_repeated_forgone)


# forgone_list
# determine which items from the recall_list where in the forgone_list
common_elements = [item for item in forgone_list if item in this_sub_recognized] 
#print(common_elements) 

print('recognition rate forgone')
recognition_forgone = len(common_elements)/len(forgone_list) if len(forgone_list) != 0 else 0
print(recognition_forgone)


recognition rate passed_fixated
0.6666666666666666
recognition rate fixated incepted
1.0
recognition rate repeated forgone
0
recognition rate forgone
0.0
